In [2]:
import pandas as pd
import numpy as np

In [72]:
mobile=pd.read_csv('Mobile.csv')

In [56]:
mobile
mobile.tail()

,Products_link,Products_Model,Products_details,Price_regular,Price_offers,Currency,Known_price
2522,https://www.device-specs.io/categories/smartph...,Apple iPhone 12,2020 - A14 Bionic Chip - 6 cores - 6.1 inch - ...,731.00,,NaN,3
2523,https://www.device-specs.io/categories/smartph...,Samsung Galaxy A53,2022 - Samsung Exynos 1280 - 8 cores - 6.5 inc...,417.45,,NaN,4
2524,https://www.device-specs.io/categories/smartph...,Samsung Galaxy A53,2022 - Samsung Exynos 1280 - 8 cores - 6.5 inc...,377.41,,NaN,3
2525,https://www.device-specs.io/categories/smartph...,Samsung Galaxy A52S,Qualcomm Snapdragon - 8 cores - 6.5 inch - 128...,364.26,,NaN,1
2526,https://www.device-specs.io/categories/smartph...,Samsung Galaxy Xcover 5,2021 - Samsung Exynos 850 - 8 cores - 5.3 inch...,271.00,263.99,EUR,3


In [57]:
mobile.shape

(2527, 7)

In [58]:
mobile['Products_link'].head()

0    https://www.device-specs.io/categories/smartph...
1    https://www.device-specs.io/categories/smartph...
2    https://www.device-specs.io/categories/smartph...
3    https://www.device-specs.io/categories/smartph...
4    https://www.device-specs.io/categories/smartph...
Name: Products_link, dtype: object

In [59]:
def data_abstract(txt1): 
    import numpy as np
    try:
        a1 = txt1['cpu']['type']
    except KeyError:
        a1=np.nan
    try:
        a2 = txt1['camera']['camera_back__mp']
    except KeyError:
        a2=np.nan
    try:
        a3 = txt1['camera']['camera_front__mp']
    except KeyError:
        a3=np.nan
    try:
        a4 = txt1['battery']['capacity__mAh']
    except KeyError:
        a4=np.nan
    try:
        a5 = txt1['display']['type']
    except KeyError:
        a5=np.nan
    try:
        a6 = txt1['display']['size__inch']
    except KeyError:
        a6=np.nan
    try:
        a7 = txt1['display']['resolution__pixels'].split('x')[0]
    except KeyError:
        a7=np.nan
    try:
        a8 = txt1['display']['resolution__pixels'].split('x')[1]
    except KeyError:
        a8=np.nan
    try:
        a9 = txt1['general']['brand']
    except KeyError:
        a9=np.nan
    try:
        a11 = txt1['software']['os']
    except KeyError:
        a11=np.nan
    try:
        a12 = txt1['measurements']['width__mm']
    except KeyError:
        a12=np.nan
    try:
        a13 = txt1['measurements']['height__mm']
    except KeyError:
        a13=np.nan
    try:
        a14 = txt1['measurements']['length__mm']
    except KeyError:
        a14=np.nan
    try:
        a10 = txt1['storage']['capacity__gb']
    except KeyError:
        a10=np.nan
    try:
        a15 = txt1['cpu']['number_of_cores']
    except KeyError:
        a15=np.nan
    try:
        a16 = txt1['ports_interfaces']['type_of_connector']
    except KeyError:
        a16=np.nan
    
    data=[a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16]
    return data

In [60]:
import ast
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd
new=[]
failed=[]
for i in range(mobile['Products_link'].size):
    try:
        html_text=requests.get(mobile['Products_link'][i]).text
        soup = BeautifulSoup(html_text,'lxml')
        code=soup.find_all('code',class_='language-json')
        code=str(code)
        pattern = re.compile("<.*?>")
        txt=pattern.sub("", code)
        pattern = re.compile("[\n ]")
        txt=pattern.sub("", txt)
        pattern = re.compile("true")
        txt=pattern.sub("True", txt)
        pattern = re.compile("false")
        txt=pattern.sub("False", txt)
        txt1=ast.literal_eval(txt)
        txt1=txt1[0]
        g=data_abstract(txt1)
        g.append(mobile['Price_regular'][i])
        g.append(mobile['Currency'][i])
        
        a=re.findall(r"\b\d{4}\b",mobile['Products_details'][i]) 
        if len(a)!=0:
            year=a[0].split()[0]
        else:
            year=np.nan
        #print(year)
        g.append(year)
        
        a=re.findall(r'\d{1,2} cores',mobile['Products_details'][i]) 
        if len(a)!=0:
            cores=a[0].split()[0]
        else:
            cores=np.nan
        #print(cores)
        g.append(cores)
        
        
        new.append(g)
    except IndexError:
        failed.append(i)
        continue
columns = ['cpu_name','camera_back__mp', 'camera_front__mp',
           'battery','display_type','display_size',
           'display_resolution_row','display_resolution_col',
           'brand','storage','os','width','height','length','cpu_cores','connector','Price','Currency','Year','Cores']
df = pd.DataFrame(data=new, columns=columns)
df

,cpu_name,camera_back__mp,camera_front__mp,battery,display_type,display_size,display_resolution_row,display_resolution_col,brand,storage,os,width,height,length,cpu_cores,connector,Price,Currency,Year,Cores
0,NaN,12.0,7.0,3174.0,SuperRetina,6.50,2560,1080,Apple,256.0,iOS10.0,NaN,NaN,NaN,6.0,AppleLightning,899.00,EUR,2018,6
1,NaN,13.0,8.0,5000.0,LCD,6.52,1600,720,OPPO,64.0,Android11,NaN,NaN,NaN,8.0,USB-C,179.38,EUR,2021,8
2,NaN,13.0,5.0,3020.0,LCD,5.71,720,1520,Huawei,16.0,Android9.0Pie,NaN,NaN,NaN,4.0,Micro-USB,144.00,NaN,2019,4
3,NaN,12.0,12.0,3500.0,OLED,6.50,2688,1242,Apple,256.0,iOS13,NaN,NaN,NaN,6.0,AppleLightning,536.00,NaN,NaN,6
4,NaN,12.0,12.0,3500.0,OLED,6.50,2688,1242,Apple,64.0,iOS13,NaN,NaN,NaN,6.0,AppleLightning,486.00,NaN,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2156,Apple,12.0,7.0,NaN,IPS,4.70,1334,750,APPLE,64.0,iOS13,67.3,138.4,7.3,NaN,Lightning,434.80,NaN,NaN,NaN
2157,MediaTek,50.0,16.0,5000.0,IPS-LCD,6.53,720,1640,Motorola,64.0,Android,75.0,164.0,8.5,8.0,USB-C,155.00,EUR,2022,8
2158,A14BionicChip,12.0,12.0,2775.0,Retina,6.10,2532,1170,Apple,64.0,iOS,71.5,146.7,7.4,6.0,AppleLightning,709.00,EUR,2020,6
2159,A14BionicChip,12.0,12.0,2775.0,Retina,6.10,2532,1170,Apple,64.0,iOS,71.5,146.7,7.4,6.0,AppleLightning,731.00,NaN,2020,6


In [64]:
df.to_csv('Mobile_data.csv', index=False)

In [66]:
ff=pd.read_csv('Mobile_data.csv')
ff.head()

,cpu_name,camera_back__mp,camera_front__mp,battery,display_type,display_size,display_resolution_row,display_resolution_col,brand,storage,os,width,height,length,cpu_cores,connector,Price,Currency,Year,Cores
0,NaN,12.0,7.0,3174.0,SuperRetina,6.50,2560.0,1080.0,Apple,256.0,iOS10.0,NaN,NaN,NaN,6.0,AppleLightning,899.00,EUR,2018.0,6.0
1,NaN,13.0,8.0,5000.0,LCD,6.52,1600.0,720.0,OPPO,64.0,Android11,NaN,NaN,NaN,8.0,USB-C,179.38,EUR,2021.0,8.0
2,NaN,13.0,5.0,3020.0,LCD,5.71,720.0,1520.0,Huawei,16.0,Android9.0Pie,NaN,NaN,NaN,4.0,Micro-USB,144.00,NaN,2019.0,4.0
3,NaN,12.0,12.0,3500.0,OLED,6.50,2688.0,1242.0,Apple,256.0,iOS13,NaN,NaN,NaN,6.0,AppleLightning,536.00,NaN,NaN,6.0
4,NaN,12.0,12.0,3500.0,OLED,6.50,2688.0,1242.0,Apple,64.0,iOS13,NaN,NaN,NaN,6.0,AppleLightning,486.00,NaN,NaN,6.0


In [68]:
ff.dtypes

cpu_name                   object
camera_back__mp           float64
camera_front__mp          float64
battery                   float64
display_type               object
display_size              float64
display_resolution_row    float64
display_resolution_col    float64
brand                      object
storage                   float64
os                         object
width                     float64
height                    float64
length                    float64
cpu_cores                 float64
connector                  object
Price                     float64
Currency                   object
Year                      float64
Cores                     float64
dtype: object

In [67]:
ff.isnull().sum()

cpu_name                  1156
camera_back__mp             75
camera_front__mp            70
battery                    319
display_type               232
display_size                 3
display_resolution_row     156
display_resolution_col     156
brand                        0
storage                      7
os                          17
width                     1178
height                    1199
length                    1176
cpu_cores                  388
connector                  248
Price                       16
Currency                  1316
Year                       828
Cores                      390
dtype: int64

In [70]:
len(failed)

366

In [73]:
import ast
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd
new1=[]
failed1=[]
for i in failed:
    try:
        html_text=requests.get(mobile['Products_link'][i]).text
        soup = BeautifulSoup(html_text,'lxml')
        code=soup.find_all('code',class_='language-json')
        code=str(code)
        pattern = re.compile("<.*?>")
        txt=pattern.sub("", code)
        pattern = re.compile("[\n ]")
        txt=pattern.sub("", txt)
        pattern = re.compile("true")
        txt=pattern.sub("True", txt)
        pattern = re.compile("false")
        txt=pattern.sub("False", txt)
        txt1=ast.literal_eval(txt)
        txt1=txt1[0]
        g=data_abstract(txt1)
        g.append(mobile['Price_regular'][i])
        g.append(mobile['Currency'][i])
        
        a=re.findall(r"\b\d{4}\b",mobile['Products_details'][i]) 
        if len(a)!=0:
            year=a[0].split()[0]
        else:
            year=np.nan
        #print(year)
        g.append(year)
        
        a=re.findall(r'\d{1,2} cores',mobile['Products_details'][i]) 
        if len(a)!=0:
            cores=a[0].split()[0]
        else:
            cores=np.nan
        #print(cores)
        g.append(cores)
        
        
        new1.append(g)
    except IndexError:
        failed1.append(i)
        continue
columns = ['cpu_name','camera_back__mp', 'camera_front__mp',
           'battery','display_type','display_size',
           'display_resolution_row','display_resolution_col',
           'brand','storage','os','width','height','length','cpu_cores','connector','Price','Currency','Year','Cores']
df1 = pd.DataFrame(data=new1, columns=columns)
df1

,cpu_name,camera_back__mp,camera_front__mp,battery,display_type,display_size,display_resolution_row,display_resolution_col,brand,storage,os,width,height,length,cpu_cores,connector,Price,Currency,Year,Cores
0,NaN,20.0,8.0,3200.0,IPS-LCD,5.15,1920,1080,Honor,64,Android7.0,NaN,NaN,NaN,8.0,USB-C,199.00,EUR,NaN,8
1,NaN,40.0,32.0,4200.0,OLED,6.47,2340,1080,Huawei,128,Android9.0Pie,NaN,NaN,NaN,8.0,USB-C,361.24,NaN,2019,8
2,NaN,40.0,24.0,4000.0,Amoled,6.10,2240,1080,Huawei,128,Android8.1,NaN,NaN,NaN,4.0,USB-C,254.00,EUR,NaN,4
3,NaN,13.0,5.0,3000.0,IPS-LCD,5.70,1440,720,Huawei,16,Android8.0,NaN,NaN,NaN,4.0,Micro-USB,200.63,NaN,2018,4
4,NaN,8.0,5.0,3020.0,IPS-LCD,5.45,720,1480,Huawei,32,Android10,NaN,NaN,NaN,8.0,Micro-USB,133.99,NaN,2020,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,A15Bionicchip,12.0,12.0,3095.0,OLED,6.10,2532,1170,Apple,128,iOS,71.5,146.7,7.7,6.0,AppleLightning,814.99,EUR,2021,6
200,QualcommSnapdragon,50.0,16.0,NaN,AMOLED,6.62,1080,2400,REALME,256,Android12,75.8,162.9,8.6,8.0,NaN,573.34,NaN,NaN,8
201,Apple,12.0,5.0,NaN,NaN,4.70,1334,750,APPLE,64,iOS11,67.1,138.3,7.1,NaN,"3,5mm",229.00,NaN,NaN,NaN
202,Apple,12.0,12.0,2775.0,Retina,6.10,2532,1170,Apple,128,iOS14,71.5,146.7,7.4,6.0,AppleLightning,999.00,EUR,2020,6


In [74]:
df1.to_csv('Mobile_data1.csv', index=False)

In [76]:
len(failed1)

162

In [77]:
import ast
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd
new2=[]
failed2=[]
for i in failed1:
    try:
        html_text=requests.get(mobile['Products_link'][i]).text
        soup = BeautifulSoup(html_text,'lxml')
        code=soup.find_all('code',class_='language-json')
        code=str(code)
        pattern = re.compile("<.*?>")
        txt=pattern.sub("", code)
        pattern = re.compile("[\n ]")
        txt=pattern.sub("", txt)
        pattern = re.compile("true")
        txt=pattern.sub("True", txt)
        pattern = re.compile("false")
        txt=pattern.sub("False", txt)
        txt1=ast.literal_eval(txt)
        txt1=txt1[0]
        g=data_abstract(txt1)
        g.append(mobile['Price_regular'][i])
        g.append(mobile['Currency'][i])
        
        a=re.findall(r"\b\d{4}\b",mobile['Products_details'][i]) 
        if len(a)!=0:
            year=a[0].split()[0]
        else:
            year=np.nan
        #print(year)
        g.append(year)
        
        a=re.findall(r'\d{1,2} cores',mobile['Products_details'][i]) 
        if len(a)!=0:
            cores=a[0].split()[0]
        else:
            cores=np.nan
        #print(cores)
        g.append(cores)
        
        
        new2.append(g)
    except IndexError:
        failed2.append(i)
        continue
columns = ['cpu_name','camera_back__mp', 'camera_front__mp',
           'battery','display_type','display_size',
           'display_resolution_row','display_resolution_col',
           'brand','storage','os','width','height','length','cpu_cores','connector','Price','Currency','Year','Cores']
df2 = pd.DataFrame(data=new2, columns=columns)
df2

,cpu_name,camera_back__mp,camera_front__mp,battery,display_type,display_size,display_resolution_row,display_resolution_col,brand,storage,os,width,height,length,cpu_cores,connector,Price,Currency,Year,Cores
0,8655G,NaN,10,4500,AMOLED,6.7,NaN,NaN,Samsung,128,Android10,NaN,NaN,NaN,NaN,NaN,1199.99,NaN,NaN,NaN
1,QualcommSnapdragon855,16.0,10,4380,AMOLED,7.3,NaN,NaN,Samsung,512,Android9.0Pie,62.74,160.78,17.02,NaN,USBTypeC,1781.99,USD,NaN,NaN


In [80]:
df2.to_csv('Mobile_data2.csv', index=False)

In [83]:
failed2

[862,
 937,
 954,
 1000,
 1069,
 1145,
 1150,
 1152,
 1158,
 1162,
 1164,
 1166,
 1167,
 1181,
 1200,
 1208,
 1214,
 1241,
 1254,
 1255,
 1258,
 1263,
 1265,
 1299,
 1313,
 1340,
 1341,
 1343,
 1344,
 1350,
 1359,
 1366,
 1383,
 1388,
 1409,
 1414,
 1416,
 1423,
 1425,
 1427,
 1436,
 1454,
 1464,
 1501,
 1506,
 1516,
 1542,
 1547,
 1589,
 1592,
 1653,
 1656,
 1661,
 1663,
 1707,
 1715,
 1750,
 1772,
 1780,
 1795,
 1811,
 1828,
 1843,
 1846,
 1876,
 1883,
 1884,
 1886,
 1896,
 1906,
 1910,
 1913,
 1924,
 1926,
 1933,
 1935,
 1946,
 1952,
 2008,
 2021,
 2032,
 2039,
 2045,
 2049,
 2054,
 2060,
 2061,
 2062,
 2067,
 2068,
 2069,
 2087,
 2103,
 2115,
 2117,
 2122,
 2126,
 2138,
 2164,
 2177,
 2182,
 2194,
 2198,
 2200,
 2219,
 2227,
 2239,
 2262,
 2271,
 2278,
 2287,
 2288,
 2289,
 2290,
 2298,
 2299,
 2310,
 2314,
 2315,
 2316,
 2317,
 2319,
 2326,
 2327,
 2331,
 2338,
 2339,
 2343,
 2357,
 2359,
 2365,
 2367,
 2388,
 2394,
 2419,
 2425,
 2436,
 2441,
 2442,
 2443,
 2445,
 2452,
 2453,
 2

In [85]:
# Open a file for writing
with open('my_file.txt', 'w') as f:
    # Write each element of the list to a new line in the file
    for element in failed2:
        f.write(str(element) + '\n')

In [146]:
df=pd.read_csv('Mobile_data.csv')
df1=pd.read_csv('Mobile_data1.csv')
df2=pd.read_csv('Mobile_data2.csv')

concatenated_df = pd.concat([df, df1, df2])
concatenated_df.reset_index(inplace=True)
concatenated_df.drop(columns=['index'],inplace=True)

In [147]:
concatenated_df

,cpu_name,camera_back__mp,camera_front__mp,battery,display_type,display_size,display_resolution_row,display_resolution_col,brand,storage,os,width,height,length,cpu_cores,connector,Price,Currency,Year,Cores
0,NaN,12.0,7.0,3174.0,SuperRetina,6.50,2560.0,1080.0,Apple,256.0,iOS10.0,NaN,NaN,NaN,6.0,AppleLightning,899.00,EUR,2018.0,6.0
1,NaN,13.0,8.0,5000.0,LCD,6.52,1600.0,720.0,OPPO,64.0,Android11,NaN,NaN,NaN,8.0,USB-C,179.38,EUR,2021.0,8.0
2,NaN,13.0,5.0,3020.0,LCD,5.71,720.0,1520.0,Huawei,16.0,Android9.0Pie,NaN,NaN,NaN,4.0,Micro-USB,144.00,NaN,2019.0,4.0
3,NaN,12.0,12.0,3500.0,OLED,6.50,2688.0,1242.0,Apple,256.0,iOS13,NaN,NaN,NaN,6.0,AppleLightning,536.00,NaN,NaN,6.0
4,NaN,12.0,12.0,3500.0,OLED,6.50,2688.0,1242.0,Apple,64.0,iOS13,NaN,NaN,NaN,6.0,AppleLightning,486.00,NaN,NaN,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2362,Apple,12.0,5.0,NaN,NaN,4.70,1334.0,750.0,APPLE,64.0,iOS11,67.10,138.30,7.10,NaN,"3,5mm",229.00,NaN,NaN,NaN
2363,Apple,12.0,12.0,2775.0,Retina,6.10,2532.0,1170.0,Apple,128.0,iOS14,71.50,146.70,7.40,6.0,AppleLightning,999.00,EUR,2020.0,6.0
2364,QualcommSnapdragon8+Gen1,50.0,4.0,4400.0,Amoled,7.60,2176.0,1812.0,Samsung,512.0,Android,130.10,155.10,6.30,8.0,USB-C,1479.00,EUR,2022.0,8.0
2365,8655G,NaN,10.0,4500.0,AMOLED,6.70,NaN,NaN,Samsung,128.0,Android10,NaN,NaN,NaN,NaN,NaN,1199.99,NaN,NaN,NaN


In [148]:
concatenated_df.dtypes

cpu_name                   object
camera_back__mp           float64
camera_front__mp          float64
battery                   float64
display_type               object
display_size              float64
display_resolution_row    float64
display_resolution_col    float64
brand                      object
storage                   float64
os                         object
width                     float64
height                    float64
length                    float64
cpu_cores                 float64
connector                  object
Price                     float64
Currency                   object
Year                      float64
Cores                     float64
dtype: object

In [149]:
type(concatenated_df['Year'][1])

numpy.float64

In [150]:
int(concatenated_df['Year'][0])

2018

In [151]:
import warnings
warnings.filterwarnings("ignore")

for i in range(concatenated_df['display_resolution_row'].size):
    try:
        concatenated_df['display_resolution_row'][i]=int(concatenated_df['display_resolution_row'][i])
    except:
        concatenated_df['display_resolution_row'][i]

        
for i in range(concatenated_df['display_resolution_col'].size):
    try:
        concatenated_df['display_resolution_col'][i]=int(concatenated_df['display_resolution_col'][i])
    except:
        concatenated_df['display_resolution_col'][i]        

        

for i in range(concatenated_df['Year'].size):
    try:
        concatenated_df['Year'][i]=int(concatenated_df['Year'][i])
    except:
        concatenated_df['Year'][i]
        
for i in range(concatenated_df['Cores'].size):
    try:
        concatenated_df['Cores'][i]=int(concatenated_df['Cores'][i])
    except:
        concatenated_df['Cores'][i]

In [336]:
concatenated_df

,cpu_name,camera_back__mp,camera_front__mp,battery,display_type,display_size,display_resolution_row,display_resolution_col,brand,storage,os,width,height,length,cpu_cores,connector,Price,Currency,Year,Cores
0,NaN,12.0,7.0,3174.0,SuperRetina,6.50,2560.0,1080.0,Apple,256.0,iOS10.0,NaN,NaN,NaN,6.0,AppleLightning,899.00,EUR,2018.0,6.0
1,NaN,13.0,8.0,5000.0,LCD,6.52,1600.0,720.0,OPPO,64.0,Android11,NaN,NaN,NaN,8.0,USB-C,179.38,EUR,2021.0,8.0
2,NaN,13.0,5.0,3020.0,LCD,5.71,720.0,1520.0,Huawei,16.0,Android9.0Pie,NaN,NaN,NaN,4.0,Micro-USB,144.00,NaN,2019.0,4.0
3,NaN,12.0,12.0,3500.0,OLED,6.50,2688.0,1242.0,Apple,256.0,iOS13,NaN,NaN,NaN,6.0,AppleLightning,536.00,NaN,NaN,6.0
4,NaN,12.0,12.0,3500.0,OLED,6.50,2688.0,1242.0,Apple,64.0,iOS13,NaN,NaN,NaN,6.0,AppleLightning,486.00,NaN,NaN,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2362,Apple,12.0,5.0,NaN,NaN,4.70,1334.0,750.0,APPLE,64.0,iOS11,67.10,138.30,7.10,NaN,"3,5mm",229.00,NaN,NaN,NaN
2363,Apple,12.0,12.0,2775.0,Retina,6.10,2532.0,1170.0,Apple,128.0,iOS14,71.50,146.70,7.40,6.0,AppleLightning,999.00,EUR,2020.0,6.0
2364,QualcommSnapdragon8+Gen1,50.0,4.0,4400.0,Amoled,7.60,2176.0,1812.0,Samsung,512.0,Android,130.10,155.10,6.30,8.0,USB-C,1479.00,EUR,2022.0,8.0
2365,8655G,NaN,10.0,4500.0,AMOLED,6.70,NaN,NaN,Samsung,128.0,Android10,NaN,NaN,NaN,NaN,NaN,1199.99,NaN,NaN,NaN


In [429]:
phone=concatenated_df.copy()

In [430]:
phone.isna().sum()

cpu_name                  1297
camera_back__mp             79
camera_front__mp            77
battery                    345
display_type               248
display_size                 3
display_resolution_row     171
display_resolution_col     171
brand                        0
storage                      7
os                          19
width                     1326
height                    1346
length                    1322
cpu_cores                  427
connector                  268
Price                       21
Currency                  1441
Year                       909
Cores                      429
dtype: int64

In [431]:
phone.drop(columns=['width','height','length','Cores','Currency'],inplace=True)

In [432]:
phone.isnull().sum()

cpu_name                  1297
camera_back__mp             79
camera_front__mp            77
battery                    345
display_type               248
display_size                 3
display_resolution_row     171
display_resolution_col     171
brand                        0
storage                      7
os                          19
cpu_cores                  427
connector                  268
Price                       21
Year                       909
dtype: int64

In [433]:
phone.shape

(2367, 15)

In [434]:
phone.head()

,cpu_name,camera_back__mp,camera_front__mp,battery,display_type,display_size,display_resolution_row,display_resolution_col,brand,storage,os,cpu_cores,connector,Price,Year
0,NaN,12.0,7.0,3174.0,SuperRetina,6.50,2560.0,1080.0,Apple,256.0,iOS10.0,6.0,AppleLightning,899.00,2018.0
1,NaN,13.0,8.0,5000.0,LCD,6.52,1600.0,720.0,OPPO,64.0,Android11,8.0,USB-C,179.38,2021.0
2,NaN,13.0,5.0,3020.0,LCD,5.71,720.0,1520.0,Huawei,16.0,Android9.0Pie,4.0,Micro-USB,144.00,2019.0
3,NaN,12.0,12.0,3500.0,OLED,6.50,2688.0,1242.0,Apple,256.0,iOS13,6.0,AppleLightning,536.00,NaN
4,NaN,12.0,12.0,3500.0,OLED,6.50,2688.0,1242.0,Apple,64.0,iOS13,6.0,AppleLightning,486.00,NaN


In [435]:
phone['brand']=phone['brand'].apply(lambda x:x.lower())

In [436]:
phone['brand'].unique()

array(['apple', 'oppo', 'huawei', 'samsung', 'xiaomi', 'motorola',
       'google', 'oneplus', 'realme', 'emporia', 'vivo', 'crosscall',
       'ulefone', 'blackview', 'simplemobile', 'iphone', 'sony',
       'blabloo', 'lg', 'poco', 'nokia', 'doro', 'unihertz', 'telekom',
       'gigaset', 'oukitel', 'honor', 'doogee', 'tcl', 'tim',
       'blackberry', 'myphone', 'asus', 'cat', 'merkloos', 'lenovo',
       'microsoft', 'fairphone', 'alcatel', 'syco', 'jbl', 'redmi',
       'hammer', 'panasonic', 'boostmobile', 'tracfone', 'blaupunkt',
       'senifone', 'vertu', 'thymos', 'fysic', 'wileyfox',
       'i.safe-mobile', 'generalmobile', 'beafon', 'hama', 'umidigi',
       'simpc', 'cyrus', 'yealink', 'nvt', 'olympia', 'ruggear',
       'googlenest', 'ordissimo', 'caterpillar', 'zte', 'cubot'],
      dtype=object)

In [437]:
phone['display_type'].unique()

array(['SuperRetina', 'LCD', 'OLED', 'DynamicAmoled', 'Amoled', 'Retina',
       nan, 'SuperRetinaXDR-display', 'IPS-LCD', 'TFT', 'IPS',
       'SuperRetinaXDRdisplay', 'AMOLED', 'TFT-LCD', 'SuperAmoled',
       'HDSuperAmoled', 'SuperAMOLED', 'DynamicAMOLED2X', 'RetinaDisplay',
       'SuperAmoledPlus', 'SuperAMOLEDPlus', 'FHDHDR', 'LTPS', 'LED',
       'Other', 'Super-LCD', 'LTPS-LCD', 'RetinaHDdisplay', 'Geenscherm',
       'FluidAMOLED', 'SAMOLED', 'PLS', 'SuperAMOLEDHD'], dtype=object)

In [438]:
phone['display_type'].replace({'Amoled':'AMOLED','SAMOLED':'SuperAMOLED'},inplace=True)

In [439]:
phone['os'].unique()

array(['iOS10.0', 'Android11', 'Android9.0Pie', 'iOS13', 'Android12',
       'iOS14', 'Anders', 'Android', 'iOS15', 'Android13', 'AppleiOS14',
       'Android10', 'iOS12', 'AppleiOS16', 'Android5.0', 'Android7.0',
       'Android8.1', 'Android7.1', 'Android5.1', 'AndroidOne', 'WearOS',
       'AppleiOS15', 'AppleiOS', 'Android8.0', 'AppleiOS13', 'Android6.0',
       'Android4.7', nan, 'Android9.0', 'Android8.1Go', 'iOS', 'iOS16',
       'Android11Go', 'KaiOS', 'AppleiOS12', 'Android10Go', 'iOS8.0',
       'iOS11', 'iOS7.0', 'Windows8.1', 'Windows10', 'Android4.4',
       'OxygenOS', 'Android10.0', 'MIUI12', 'Android10Goedition',
       'Android8.1Goedition', 'LinuxUbuntu', 'Android8.1Oreo'],
      dtype=object)

In [440]:
phone['os'].replace({'AppleiOS14':'iOS14','AppleiOS16':'iOS16','AppleiOS15':'iOS15','AppleiOS13':'iOS13','AppleiOS12':'iOS12','AppleiOS':
                    'iOS'},inplace=True)

In [447]:
a=phone.loc[phone['camera_back__mp'].isna()]['camera_front__mp'].unique()
b=[]
for i in a:
    if i==i:
        b.append(i)
b
for i in b:
    try:
        d=phone.loc[(phone['camera_front__mp']==i)][['camera_back__mp']].value_counts().idxmax()[0]
    except:
        d=np.nan
    phone.loc[(phone['camera_front__mp']==i)&(phone['camera_back__mp'].isna()),'camera_back__mp']=d

In [448]:
a=phone.loc[phone['camera_front__mp'].isna()]['camera_back__mp'].unique()
b=[]
for i in a:
    if i==i:
        b.append(i)
for i in b:
    try:
        d=phone.loc[(phone['camera_back__mp']==i)][['camera_front__mp']].value_counts().idxmax()[0]
    except:
        d=np.nan
    phone.loc[(phone['camera_back__mp']==i)&(phone['camera_front__mp'].isna()),'camera_front__mp']=d

In [451]:
a=phone.loc[phone['battery'].isna()]['Price'].unique()
b=[]
for i in a:
    if i==i:
        b.append(i)
b
for i in b:
    try:
        d=phone.loc[(phone['Price']==i)][['battery']].value_counts().idxmax()[0]
    except:
        d=np.nan
    phone.loc[(phone['Price']==i)&(phone['battery'].isna()),'battery']=d

In [457]:
a=phone.loc[phone['display_resolution_row'].isna()]['display_resolution_col'].unique()
b=[]
for i in a:
    if i==i:
        b.append(i)
b
for i in b:
    try:
        d=phone.loc[(phone['display_resolution_col']==i)][['display_resolution_row']].value_counts().idxmax()[0]
    except:
        d=np.nan
    phone.loc[(phone['display_resolution_col']==i)&(phone['display_resolution_row'].isna()),'display_resolution_row']=d

In [458]:
a=phone.loc[phone['display_resolution_col'].isna()]['display_resolution_row'].unique()
b=[]
for i in a:
    if i==i:
        b.append(i)
b
for i in b:
    try:
        d=phone.loc[(phone['display_resolution_row']==i)][['display_resolution_col']].value_counts().idxmax()[0]
    except:
        d=np.nan
    phone.loc[(phone['display_resolution_row']==i)&(phone['display_resolution_col'].isna()),'display_resolution_col']=d

In [463]:
a=phone.loc[phone['display_type'].isna()]['display_size'].unique()
b=[]
for i in a:
    if i==i:
        b.append(i)
b
for i in b:
    try:
        d=phone.loc[(phone['display_size']==i)][['display_type']].value_counts().idxmax()[0]
    except:
        d=np.nan
    phone.loc[(phone['display_size']==i)&(phone['display_type'].isna()),'display_type']=d

In [464]:
a=phone.loc[phone['display_size'].isna()]['display_type'].unique()
b=[]
for i in a:
    if i==i:
        b.append(i)
b
for i in b:
    try:
        d=phone.loc[(phone['display_type']==i)][['display_size']].value_counts().idxmax()[0]
    except:
        d=np.nan
    phone.loc[(phone['display_type']==i)&(phone['display_size'].isna()),'display_size']=d

In [467]:
a=phone.loc[phone['storage'].isna()]['Price'].unique()
b=[]
for i in a:
    if i==i:
        b.append(i)
b
for i in b:
    try:
        d=phone.loc[(phone['Price']==i)][['storage']].value_counts().idxmax()[0]
    except:
        d=np.nan
    phone.loc[(phone['Price']==i)&(phone['storage'].isna()),'storage']=d

In [468]:
a=phone.loc[phone['Price'].isna()]['storage'].unique()
b=[]
for i in a:
    if i==i:
        b.append(i)
b
for i in b:
    try:
        d=phone.loc[(phone['storage']==i)][['Price']].value_counts().idxmax()[0]
    except:
        d=np.nan
    phone.loc[(phone['storage']==i)&(phone['Price'].isna()),'Price']=d

In [469]:
a=phone.loc[phone['cpu_cores'].isna()]['Price'].unique()
b=[]
for i in a:
    if i==i:
        b.append(i)
b
for i in b:
    try:
        d=phone.loc[(phone['Price']==i)][['cpu_cores']].value_counts().idxmax()[0]
    except:
        d=np.nan
    phone.loc[(phone['Price']==i)&(phone['cpu_cores'].isna()),'cpu_cores']=d

In [473]:
a=phone.loc[phone['Year'].isna()]['cpu_name'].unique()
b=[]
for i in a:
    if i==i:
        b.append(i)
b
for i in b:
    try:
        d=phone.loc[(phone['cpu_name']==i)][['Year']].value_counts().idxmax()[0]
    except:
        d=np.nan
    phone.loc[(phone['cpu_name']==i)&(phone['Year'].isna()),'Year']=d

In [474]:
a=phone.loc[phone['os'].isna()]['brand'].unique()
b=[]
for i in a:
    if i==i:
        b.append(i)
b
for i in b:
    try:
        d=phone.loc[(phone['brand']==i)][['os']].value_counts().idxmax()[0]
    except:
        d=np.nan
    phone.loc[(phone['brand']==i)&(phone['os'].isna()),'os']=d

In [475]:
a=phone.loc[phone['connector'].isna()]['brand'].unique()
b=[]
for i in a:
    if i==i:
        b.append(i)
b
for i in b:
    try:
        d=phone.loc[(phone['brand']==i)][['connector']].value_counts().idxmax()[0]
    except:
        d=np.nan
    phone.loc[(phone['brand']==i)&(phone['connector'].isna()),'connector']=d

In [478]:
z=(phone['camera_back__mp'].isna())|(phone['camera_front__mp'].isna())|(phone['battery'].isna())|(phone['display_resolution_row'].isna())|(phone['display_resolution_col'].isna())|(phone['cpu_cores'].isna()|(phone['connector'].isna()))
phone=phone[~z]

In [479]:
phone.shape

(1832, 15)

In [480]:
phone.isna().sum()

cpu_name                  1261
camera_back__mp              0
camera_front__mp             0
battery                      0
display_type                 0
display_size                 0
display_resolution_row       0
display_resolution_col       0
brand                        0
storage                      0
os                           0
cpu_cores                    0
connector                    0
Price                        0
Year                       424
dtype: int64

In [481]:
phone['Year'].replace(np.nan,'Unknown',inplace=True)

In [482]:
phone.isna().sum()

cpu_name                  1261
camera_back__mp              0
camera_front__mp             0
battery                      0
display_type                 0
display_size                 0
display_resolution_row       0
display_resolution_col       0
brand                        0
storage                      0
os                           0
cpu_cores                    0
connector                    0
Price                        0
Year                         0
dtype: int64